# IT Service Ticket Classification

## Objetivo

Desenvolver um sistema de classificação automática de tickets de suporte de TI que:
- **Entrada:** texto do ticket (string)
- **Saída:** `{"classe": "...", "justificativa": "..."}`

## Dataset

Utilizamos o dataset [IT Service Ticket Classification](https://www.kaggle.com/datasets/adisongoh/it-service-ticket-classification-dataset), que contém ~48.000 tickets de suporte de TI rotulados em 8 categorias.

---

# 1. Análise Exploratória dos Dados

Antes de implementar o sistema de classificação, é essencial compreender a estrutura dos dados, a distribuição das classes e as características dos textos.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from collections import Counter

## 1.1 Carregamento dos Dados

In [ ]:
df = pd.read_csv("dataset.csv")
print(f"Total de tickets: {len(df):,}")
print(f"Colunas: {list(df.columns)}")
df.head(10)

O dataset possui duas colunas:
- **Document**: O texto do ticket de suporte
- **Topic_group**: A categoria/classe do ticket

## 1.2 Qualidade dos Dados

In [ ]:
print("Informações do DataFrame:")
print("-" * 40)
df.info()

In [ ]:
print("Valores nulos por coluna:")
print(df.isnull().sum())
print(f"\nTotal de valores nulos: {df.isnull().sum().sum()}")

In [ ]:
print("Valores duplicados:")
duplicates = df.duplicated().sum()
print(f"Tickets duplicados: {duplicates} ({duplicates/len(df)*100:.2f}%)")

## 1.3 Distribuição das Classes

In [ ]:
class_counts = df["Topic_group"].value_counts().reset_index()
class_counts.columns = ["Classe", "Quantidade"]
class_counts["Percentual"] = (class_counts["Quantidade"] / len(df) * 100).round(2)
class_counts

In [ ]:
fig = px.bar(
    class_counts,
    x="Classe",
    y="Quantidade",
    color="Classe",
    title="Distribuição de Tickets por Classe",
    text="Quantidade",
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_traces(textposition="outside")
fig.update_layout(
    xaxis_title="Classe",
    yaxis_title="Quantidade de Tickets",
    showlegend=False,
    height=500
)
fig.show()

In [ ]:
fig = px.pie(
    class_counts,
    values="Quantidade",
    names="Classe",
    title="Proporção de Tickets por Classe",
    color_discrete_sequence=px.colors.qualitative.Set2,
    hole=0.3
)
fig.update_traces(textposition="inside", textinfo="percent+label")
fig.update_layout(height=500)
fig.show()

#### Observações sobre a distribuição

- Dataset **desbalanceado**: Hardware e HR Support representam ~51% dos dados
- As classes menores (Administrative rights, Internal Project) têm menos de 5% cada
- Total de **8 classes** distintas para classificação

#### Implicações para o modelo de classificação

1. **Amostragem para avaliação**: Para avaliar em 200 tickets, devemos usar amostragem estratificada para garantir representação de todas as classes.

2. **RAG e exemplos**: O retriever terá mais exemplos de classes majoritárias. Isso pode ser benéfico (mais contexto) ou problemático (viés).

## 1.4 Análise do Texto dos Tickets

In [ ]:
df["text_length"] = df["Document"].str.len()
df["word_count"] = df["Document"].str.split().str.len()

print("Estatísticas da contagem de palavras:")
print(df["word_count"].describe().round(2))

In [ ]:
fig = px.histogram(
    df,
    x="word_count",
    nbins=50,
    title="Distribuição da Quantidade de Palavras por Ticket",
    color_discrete_sequence=["#66c2a5"]
)
fig.update_layout(
    xaxis_title="Quantidade de Palavras",
    yaxis_title="Frequência",
    height=400
)
fig.show()

In [ ]:
# Percentis para entender melhor a distribuição
percentiles = [50, 75, 90, 95, 99]
print("Percentis de palavras por ticket:")
for p in percentiles:
    value = df["word_count"].quantile(p/100)
    print(f"  {p}%: {value:.0f} palavras")

### Discussão: Distribuição do Tamanho dos Textos

A distribuição apresenta assimetria positiva:
- A mediana é significativamente menor que a média, indicando que a maioria dos tickets é curta
- Existem outliers com textos muito longos

In [ ]:
fig = px.box(
    df,
    x="Topic_group",
    y="word_count",
    color="Topic_group",
    title="Distribuição de Palavras por Classe",
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_layout(
    xaxis_title="Classe",
    yaxis_title="Quantidade de Palavras",
    showlegend=False,
    height=500
)
fig.show()

In [ ]:
# Estatísticas por classe
stats_by_class = df.groupby("Topic_group")["word_count"].agg(["mean", "median", "std"]).round(1)
stats_by_class = stats_by_class.sort_values("mean", ascending=False)
stats_by_class

## 1.5 Análise de Palavras Frequentes

In [ ]:
all_words = " ".join(df["Document"]).lower().split()
word_freq = Counter(all_words)

print(f"Vocabulário total: {len(word_freq):,} palavras únicas")
print(f"Total de palavras: {len(all_words):,}")

top_words = pd.DataFrame(word_freq.most_common(30), columns=["Palavra", "Frequência"])
top_words

In [ ]:
fig = px.bar(
    top_words,
    x="Frequência",
    y="Palavra",
    orientation="h",
    title="Top 30 Palavras Mais Frequentes",
    color="Frequência",
    color_continuous_scale="Viridis"
)
fig.update_layout(
    yaxis={"categoryorder": "total ascending"},
    height=700,
    showlegend=False
)
fig.show()

## 1.6 Palavras Frequentes por Classe

In [ ]:
def get_top_words_by_class(df, class_name, n=10):
    class_text = " ".join(df[df["Topic_group"] == class_name]["Document"]).lower().split()
    return Counter(class_text).most_common(n)

classes = df["Topic_group"].unique()
top_words_by_class = []

for cls in classes:
    for word, freq in get_top_words_by_class(df, cls, 10):
        top_words_by_class.append({"Classe": cls, "Palavra": word, "Frequência": freq})

df_top_words = pd.DataFrame(top_words_by_class)

In [ ]:
fig = px.bar(
    df_top_words,
    x="Frequência",
    y="Palavra",
    color="Classe",
    facet_col="Classe",
    facet_col_wrap=4,
    orientation="h",
    title="Top 10 Palavras por Classe",
    color_discrete_sequence=px.colors.qualitative.Set2,
    height=800
)
fig.update_layout(showlegend=False)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

## 1.7 Exemplos de Tickets por Classe

In [ ]:
for cls in sorted(df["Topic_group"].unique()):
    count = len(df[df["Topic_group"] == cls])
    pct = count / len(df) * 100
    print(f"\n{'='*80}")
    print(f"CLASSE: {cls.upper()} ({count:,} tickets - {pct:.1f}%)")
    print("="*80)
    samples = df[df["Topic_group"] == cls].sample(2, random_state=42)
    for i, (_, row) in enumerate(samples.iterrows(), 1):
        text = row['Document']
        truncated = text[:300] + "..." if len(text) > 300 else text
        print(f"\n[Exemplo {i}]")
        print(truncated)

## 1.8 Resumo da Análise Exploratória

In [ ]:
print("=" * 60)
print("RESUMO DA ANÁLISE")
print("=" * 60)

print(f"\n[Dataset]")
print(f"  Total de tickets: {len(df):,}")
print(f"  Número de classes: {df['Topic_group'].nunique()}")
print(f"  Sem valores nulos ou duplicados")

print(f"\n[Texto]")
print(f"  Média de palavras: {df['word_count'].mean():.1f}")
print(f"  Mediana de palavras: {df['word_count'].median():.1f}")
print(f"  Máximo de palavras: {df['word_count'].max()}")

print(f"\n[Distribuição das Classes]")
for _, row in class_counts.iterrows():
    print(f"  {row['Classe']}: {row['Quantidade']:,} ({row['Percentual']}%)")

### Conclusões e Escolha da Abordagem

Com base na análise exploratória, identificamos características que guiam a escolha da abordagem de classificação:

**Características do dataset:**
- **~48k tickets rotulados** - Dataset grande com labels confiáveis
- **8 classes com desbalanceamento** - Classes minoritárias (Administrative rights ~3.7%) precisam de tratamento especial
- **Textos curtos** - Mediana de ~26 palavras permite processar múltiplos exemplos no contexto
- **Padrões de vocabulário distintos** - Cada classe tem palavras-chave características (ex: "card" para Access, "po/purchase" para Purchase)

**Abordagem escolhida: RAG + LLM**

Utilizamos **RAG (Retrieval Augmented Generation)** combinado com um **LLM (Large Language Model)**:

1. **Retrieval:** Buscar tickets similares no dataset de treino usando embeddings semânticos
2. **Augmented:** Enriquecer o prompt com exemplos relevantes
3. **Generation:** LLM classifica e justifica baseado no contexto

**Por que RAG?**

| Alternativa | Limitação |
|-------------|-----------|
| Fine-tuning de LLM | Requer recursos computacionais significativos e risco de overfitting |
| ML tradicional (TF-IDF + SVM) | Boa precisão, mas não gera justificativas naturais |
| Zero-shot com LLM | Perde a riqueza dos exemplos reais do dataset |

**Vantagens do RAG para este problema:**

1. **Aproveita o dataset grande:** Os ~48k tickets rotulados fornecem exemplos relevantes para qualquer novo ticket
2. **Lida com desbalanceamento:** Classes minoritárias se beneficiam de exemplos específicos recuperados
3. **Textos curtos facilitam:** A mediana de 26 palavras permite incluir vários exemplos no prompt sem exceder limites
4. **Justificativas de qualidade:** LLM vê exemplos reais e pode referenciar padrões similares
5. **Explicável:** Podemos inspecionar quais exemplos influenciaram cada decisão

---

# 2. Carregamento e Preparação dos Dados

### Amostragem Balanceada

Para avaliação, utilizamos **amostragem balanceada** em vez de estratificada:

| Tipo | Descrição |
|------|-----------|
| **Estratificada** | Mantém a proporção original (classes raras têm poucas amostras) |
| **Balanceada** | Mesmo número de amostras por classe |

**Por que balanceada?**
1. **Classes minoritárias importam** - Um classificador que falha em classes raras não é aceitável
2. **Métricas confiáveis** - 25 amostras por classe permitem avaliação estatisticamente significativa

In [ ]:
from classifier.data import load_dataset, train_test_split_balanced

In [ ]:
# Carregar dataset
df, classes = load_dataset()
print(f"Total de tickets: {len(df):,}")
df.head()

In [ ]:
# Classes obtidas do dataset
print(f"Classes ({len(classes)}):")
for c in classes:
    print(f"  - {c}")

In [ ]:
from classifier.config import TEST_SIZE

# Split balanceado: treino para RAG, teste para avaliação
# Amostragem balanceada garante o mesmo número de tickets por classe (200 / 8 = 25)
train_df, test_df = train_test_split_balanced(df, test_size=TEST_SIZE)

print(f"Treino: {len(train_df):,} tickets")
print(f"Teste:  {len(test_df)} tickets")
print(f"\nDistribuição no teste:")
print(test_df["Topic_group"].value_counts().sort_index())

## 3. RAG - Retrieval de Tickets Similares

### Como funciona

1. **Embeddings:** Cada ticket é convertido em um vetor de 384 dimensões usando o modelo `all-MiniLM-L6-v2` (sentence-transformers). Este modelo foi treinado para capturar similaridade semântica.

2. **Similaridade de cosseno:** Para encontrar tickets similares, calculamos o cosseno do ângulo entre vetores. Valores próximos de 1 indicam alta similaridade.

3. **Tickets representativos:** Para cada classe, calculamos o "centróide" (média dos embeddings) e selecionamos o ticket mais próximo. Isso nos dá um exemplo típico de cada classe.

### Componentes

- `TicketRetriever.index()`: Gera embeddings para todos os tickets de treino
- `TicketRetriever.retrieve()`: Busca os K tickets mais similares
- `TicketRetriever.compute_representatives()`: Calcula tickets representativos por classe

In [ ]:
from classifier.rag import TicketRetriever

In [ ]:
# Indexar tickets de treino
retriever = TicketRetriever()
retriever.index(train_df)

# Calcular tickets representativos de cada classe (centróides)
representatives = retriever.compute_representatives()
print(f"\nTickets representativos calculados para {len(representatives)} classes")

In [ ]:
# Testar retrieval com um ticket do conjunto de teste
test_ticket = test_df.iloc[1]
query = test_ticket["Document"]
true_class = test_ticket["Topic_group"]

similar = retriever.retrieve(query, k=5)

print(f"Ticket de teste (classe real: {true_class}):")
print(f"{query}\n")
print("Tickets similares recuperados:")
for i, ticket in enumerate(similar, 1):
    print(f"\n{i}. [{ticket['class']}] (score: {ticket['score']:.3f})")
    print(f"   {ticket['text']}")

In [ ]:
# Visualizar tickets representativos
print("Tickets representativos (mais próximos do centróide de cada classe):\n")
for class_name in sorted(representatives.keys()):
    t = representatives[class_name]
    print(f"[{class_name}] (score: {t['score']:.3f})")
    print(f"   {t['text']}\n")

## 4. Design dos Prompts

O prompt é a interface entre nosso sistema e a LLM. Um bom design de prompt é crucial para obter classificações precisas e justificativas úteis.

### Estrutura do Prompt

O prompt é dividido em duas partes:

1. **System prompt:** Define o papel da LLM (classificador), lista as classes válidas e especifica o formato de saída (JSON)

2. **User prompt:** Contém o ticket a classificar e os exemplos de contexto

### Parâmetros Configuráveis

| Parâmetro | Descrição | Valor Padrão |
|-----------|-----------|---------------|
| `K_SIMILAR` | Número de tickets similares do RAG | 5 |
| `reference_tickets` | Tickets representativos por classe | 1 por classe |

### Trade-offs

- **Mais exemplos similares:** Melhor contexto para classificação, mas aumenta tokens e custo
- **Tickets de referência:** Garante diversidade de classes, essencial para justificativas comparativas
- **Muitos exemplos:** Pode "poluir" o contexto e confundir a LLM

Vamos visualizar como os prompts são gerados:

In [ ]:
from classifier.prompts import build_system_prompt, build_user_prompt

In [ ]:
# Exemplo 1: System Prompt
system_prompt = build_system_prompt(classes)
print("=== SYSTEM PROMPT ===\n")
print(system_prompt)

In [ ]:
# Exemplo 2: User Prompt COM tickets de referência
user_prompt_with_refs = build_user_prompt(query, similar, representatives)
print("=== USER PROMPT (com tickets de referência) ===\n")
print(user_prompt_with_refs)

In [ ]:
# Exemplo 3: User Prompt SEM tickets de referência
# Útil quando queremos usar menos tokens ou quando os similares já são suficientes
user_prompt_no_refs = build_user_prompt(query, similar, reference_tickets=None)
print("=== USER PROMPT (sem tickets de referência) ===\n")
print(user_prompt_no_refs)

### Comparação de Uso de Tokens

A tabela abaixo mostra o impacto real de cada parâmetro no consumo de tokens. Isso ajuda a escolher a configuração ideal considerando o trade-off entre qualidade do contexto e custo/latência.

In [ ]:
import tiktoken

# Usar tokenizer cl100k_base (compatível com GPT-4, GPT-3.5-turbo, etc.)
enc = tiktoken.get_encoding("cl100k_base")

def count_tokens(text: str) -> int:
    """Conta tokens usando o tokenizer cl100k_base."""
    return len(enc.encode(text))

# Comparar diferentes configurações de prompt
# K=5 sem refs é a baseline (configuração mínima recomendada)
configs = [
    ("K=5, sem refs", build_user_prompt(query, similar[:5], None)),
    ("K=5, com refs", build_user_prompt(query, similar[:5], representatives)),
    ("K=3, sem refs", build_user_prompt(query, similar[:3], None)),
    ("K=3, com refs", build_user_prompt(query, similar[:3], representatives)),
    ("K=1, sem refs", build_user_prompt(query, similar[:1], None)),
    ("K=1, com refs", build_user_prompt(query, similar[:1], representatives)),
]

# Tabela comparativa
system_tokens = count_tokens(system_prompt)
print(f"System prompt: {system_tokens} tokens (fixo)\n")
print(f"{'Configuração':<16} | {'User Prompt':>12} | {'Total':>8} | {'vs baseline':>12}")
print("-" * 58)
baseline = None
for name, prompt in configs:
    user_tokens = count_tokens(prompt)
    total = system_tokens + user_tokens
    if baseline is None:
        baseline = total
        diff = "(base)"
    else:
        diff = f"{(total - baseline) / baseline * 100:+.0f}%"
    print(f"{name:<16} | {user_tokens:>12} | {total:>8} | {diff:>12}")

## 5. Classificação com LLM

Com o prompt construído, enviamos para a LLM. O sistema suporta qualquer API compatível com OpenAI configurada via variáveis de ambiente.

**Requer:** variáveis de ambiente `LLM_BASE_URL` e `LLM_MODEL` (ver `.env.example`)

In [ ]:
from classifier.llm import TicketClassifier

In [ ]:
# Inicializar classificador
classifier = TicketClassifier()

In [ ]:
# Classificar o ticket de teste
details = classifier.classify(query, similar, classes, reference_tickets=representatives)

print(f"Classe real: {true_class}")
print(f"Classe predita: {details.result.classe}")
print(f"Justificativa: {details.result.justificativa}")
if details.reasoning:
    print(f"\nReasoning:")
    print(details.reasoning)
print(f"\nCorreto: {details.result.classe == true_class}")

#### Exemplo de Resposta com Reasoning

**Sem reasoning:**
```json
{
  "classe": "Access",
  "justificativa": "O ticket menciona problemas com cartão de acesso."
}
```

**Com reasoning:**
```json
{
  "classe": "Access",
  "justificativa": "O ticket descreve problemas com 'card key' e 'door access'...",
  "reasoning": "Analisei o ticket e identifiquei palavras-chave como 'card key', 'door' e 'access'..."
}
```

#### Configuração

```bash
# .env
LLM_MODEL=xiaomi/mimo-v2-flash:free
LLM_REASONING_EFFORT=medium  # low, medium, ou high
```

Via CLI:
```bash
uv run python main.py --reasoning medium
```

### Uso de Reasoning

Ativamos o **modo reasoning** do modelo (`LLM_REASONING_EFFORT=medium`) para melhorar a qualidade das classificações.

#### Benefícios

| Aspecto | Impacto do Reasoning |
|---------|---------------------|
| **Casos ambíguos** | O modelo "pensa" antes de decidir, analisando nuances |
| **Justificativas** | Explicações mais detalhadas do processo de decisão |
| **Precisão** | Reduz erros entre classes similares |
| **Transparência** | Campo `reasoning` mostra o raciocínio completo |

#### Trade-offs

Ao ativar reasoning, enfrentamos:
| Aspecto | Trade-off |
|---------|-----------|
| **Tokens** | Aumento no número de tokens e custo por chamada |
| **Latência** | Respostas mais lentas devido ao processamento adicional |

## 5.5 Escolha do Modelo LLM

- Disponível gratuitamente via OpenRouter
- Suporte nativo a reasoning
- Performance competitiva

### Arquitetura da Solução

O diagrama abaixo ilustra o fluxo completo de classificação:

In [ ]:
import plotly.graph_objects as go

# Criar diagrama de arquitetura
fig = go.Figure()

# Definir as etapas do pipeline
steps = [
    ("Ticket", "#e1f5fe", "Texto de entrada"),
    ("Embedding", "#fff3e0", "all-MiniLM-L6-v2\n384 dimensões"),
    ("Retrieval", "#e8f5e9", "K similares +\nRepresentativos"),
    ("Prompt", "#fce4ec", "System + User\n+ Contexto RAG"),
    ("LLM", "#f3e5f5", "API OpenAI-compatible\n+ Retry JSON"),
    ("Output", "#e0f2f1", '{"classe": "...",\n"justificativa": "..."}'),
]

# Posições
x_positions = list(range(len(steps)))
y_pos = 0.5

# Adicionar caixas e textos
for i, (name, color, desc) in enumerate(steps):
    # Caixa
    fig.add_shape(
        type="rect",
        x0=i - 0.4, x1=i + 0.4,
        y0=0.2, y1=0.8,
        fillcolor=color,
        line=dict(color="#333", width=2),
    )
    # Nome da etapa
    fig.add_annotation(
        x=i, y=0.65,
        text=f"<b>{name}</b>",
        showarrow=False,
        font=dict(size=14),
    )
    # Descrição
    fig.add_annotation(
        x=i, y=0.38,
        text=desc,
        showarrow=False,
        font=dict(size=10),
        align="center",
    )
    # Seta para próxima etapa
    if i < len(steps) - 1:
        fig.add_annotation(
            x=i + 0.5, y=0.5,
            ax=i + 0.42, ay=0.5,
            xref="x", yref="y",
            axref="x", ayref="y",
            showarrow=True,
            arrowhead=2,
            arrowsize=1.5,
            arrowcolor="#333",
        )

fig.update_layout(
    title=dict(text="Arquitetura do Pipeline RAG", x=0.5, font=dict(size=16)),
    xaxis=dict(visible=False, range=[-0.6, len(steps) - 0.4]),
    yaxis=dict(visible=False, range=[0, 1]),
    height=250,
    margin=dict(l=20, r=20, t=50, b=20),
    plot_bgcolor="white",
)

fig.show()

## 6. Pipeline Completo com LangGraph

O LangGraph orquestra todo o fluxo de classificação em um grafo de estados com 4 nós:

| Nó | Função |
|----|--------|
| **embed** | Gera embedding do ticket (384 dimensões) |
| **retrieve** | Busca K tickets similares usando o embedding |
| **build_prompt** | Constrói system e user prompts com contexto RAG |
| **classify** | Chama a LLM e processa a resposta JSON |

Abaixo visualizamos a estrutura do grafo:

In [ ]:
from IPython.display import Image, display
from classifier.graph import create_graph

# Criar o grafo para visualização
pipeline = create_graph(retriever, classifier, classes, representatives)

# Visualizar a estrutura do grafo LangGraph
display(Image(pipeline.get_graph().draw_mermaid_png()))

In [ ]:
from classifier.graph import classify_ticket

In [ ]:
# Classificar usando o pipeline completo
# A função classify_ticket encapsula todo o fluxo: retrieve → classify
details_graph = classify_ticket(
    ticket=query,
    retriever=retriever,
    classifier=classifier,
    classes=classes,
    reference_tickets=representatives,
)

print(f"Classe real: {true_class}")
print(f"Classe predita: {details_graph.result.classe}")
print(f"Justificativa: {details_graph.result.justificativa}")
if details_graph.reasoning:
    print(f"\nReasoning:")
    print(details_graph.reasoning)
print(f"\nCorreto: {details_graph.result.classe == true_class}")

## 7. Avaliação do Classificador

Agora vamos avaliar o desempenho do classificador nos 200 tickets de teste.

### Métricas Utilizadas

- **Accuracy:** Proporção de classificações corretas
- **F1 Macro:** Média não-ponderada do F1 por classe (trata todas as classes igualmente, importante para datasets desbalanceados)
- **Cohen's Kappa:** Mede concordância além do acaso (valores próximos de 1 indicam excelente concordância)
- **MCC (Matthews Correlation Coefficient):** Métrica robusta para classificação multi-classe
- **Confusion Matrix:** Visualiza erros de classificação entre classes

### Tratamento de Erros

LLMs podem ocasionalmente retornar JSON malformado, ou a API pode apresentar erros. Para lidar com isso:

**Erros de JSON (parsing):**
1. **Primeira tentativa:** Envia o prompt normal para a LLM
2. **Se JSON inválido:** Continua a conversa adicionando a resposta do assistant e um prompt de correção:
   ```
   Sua resposta anterior não está no formato JSON válido.
   Por favor, responda APENAS com JSON válido no formato:
   {"classe": "<categoria>", "justificativa": "<explicação>"}
   ```
3. **Segunda tentativa:** LLM recebe o contexto completo da conversa e tenta corrigir
4. **Se falhar novamente:** O ticket é marcado como erro e reportado separadamente

**Erros de API:**
- Erros de autenticação (401), rate limit (429), ou outros erros da API são capturados
- O ticket é marcado como erro com o motivo específico
- A avaliação continua com os próximos tickets

Este approach garante que:
- Um erro pontual não interrompe toda a avaliação
- A LLM tem chance de se corrigir com contexto adicional
- Erros persistentes são documentados para análise posterior

In [ ]:
from classifier.metrics import evaluate, print_report, plot_confusion_matrix, plot_per_class_metrics
from classifier.llm import ClassificationError
from tqdm import tqdm

In [ ]:
# Classificar todos os tickets de teste
y_true = []
y_pred = []
results = []
classification_errors = []
total_tokens = {"prompt": 0, "completion": 0, "total": 0}

print(f"Classificando {len(test_df)} tickets de teste...\n")

for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Classificando"):
    ticket_text = row["Document"]
    true_label = row["Topic_group"]
    
    try:
        details = classify_ticket(
            ticket=ticket_text,
            retriever=retriever,
            classifier=classifier,
            classes=classes,
            reference_tickets=representatives,
        )
        
        y_true.append(true_label)
        y_pred.append(details.result.classe)
        results.append({
            "ticket": ticket_text,
            "true": true_label,
            "pred": details.result.classe,
            "justificativa": details.result.justificativa,
            "correct": true_label == details.result.classe,
            "token_usage": {
                "prompt_tokens": details.token_usage.prompt_tokens,
                "completion_tokens": details.token_usage.completion_tokens,
                "total_tokens": details.token_usage.total_tokens,
            },
            "similar_tickets": details.similar_tickets,
        })
        total_tokens["prompt"] += details.token_usage.prompt_tokens
        total_tokens["completion"] += details.token_usage.completion_tokens
        total_tokens["total"] += details.token_usage.total_tokens
    except ClassificationError as e:
        classification_errors.append({
            "ticket": ticket_text,
            "true": true_label,
            "reason": e.reason,
            "raw_response": e.raw_response,
        })

print(f"\nClassificação concluída!")
print(f"Classificados com sucesso: {len(results)}")
if classification_errors:
    print(f"Erros de classificação (JSON inválido): {len(classification_errors)}")

print(f"\n{'='*60}")
print("TOKEN USAGE SUMMARY")
print(f"{'='*60}")
print(f"Prompt tokens:     {total_tokens['prompt']:,}")
print(f"Completion tokens: {total_tokens['completion']:,}")
print(f"Total tokens:      {total_tokens['total']:,}")

In [ ]:
# Calcular métricas
metrics = evaluate(y_true, y_pred, classes)
print_report(metrics, classes)

### Métricas por Classe

O gráfico abaixo mostra precision, recall e F1-score para cada classe, facilitando a identificação visual de quais classes têm melhor/pior desempenho.

In [ ]:
# Gráfico de métricas por classe
plot_per_class_metrics(y_true, y_pred, classes, title="Precision, Recall e F1-Score por Classe")

In [ ]:
# Visualizar confusion matrix
plot_confusion_matrix(metrics["confusion_matrix"], classes, title="Confusion Matrix - Classificador RAG")

### Confusion Matrix Normalizada

A versão normalizada mostra percentagens em vez de contagens absolutas, facilitando a identificação de taxas de erro por classe.

In [ ]:
# Confusion matrix normalizada (percentagens)
plot_confusion_matrix(
    metrics["confusion_matrix"], 
    classes, 
    title="Confusion Matrix - Normalizada por True Label",
    normalize=True
)

### Análise de Erros

Existem dois tipos de erros na avaliação:

1. **Erros de classificação:** A LLM retornou um JSON válido, mas a classe predita difere da classe real
2. **Erros de parsing:** A LLM não retornou JSON válido mesmo após retry (tickets não incluídos nas métricas)

Abaixo analisamos os erros de classificação para entender os padrões de confusão entre classes.

In [ ]:
# Filtrar erros
errors = [r for r in results if not r["correct"]]
print(f"Total de erros: {len(errors)} de {len(results)} ({100 * len(errors) / len(results):.1f}%)\n")

# Mostrar alguns exemplos de erros
print("=" * 80)
print("EXEMPLOS DE ERROS DE CLASSIFICAÇÃO")
print("=" * 80)

for i, error in enumerate(errors[:5], 1):
    print(f"\n--- Erro {i} ---")
    print(f"Classe real:     {error['true']}")
    print(f"Classe predita:  {error['pred']}")
    print(f"Justificativa:   {error['justificativa']}")
    print(f"Ticket:          {error['ticket'][:200]}...")

In [ ]:
# Distribuição de erros por classe
from collections import Counter

print("Distribuição de erros por classe real:")
error_by_true = Counter(e["true"] for e in errors)
for cls, count in sorted(error_by_true.items(), key=lambda x: -x[1]):
    print(f"  {cls}: {count} erros")

print("\nConfusões mais comuns (real → predito):")
confusion_pairs = Counter((e["true"], e["pred"]) for e in errors)
for (true, pred), count in confusion_pairs.most_common(10):
    print(f"  {true} → {pred}: {count}x")

### Distribuição de Erros por Classe

Visualização gráfica dos erros por classe real, facilitando a identificação de quais classes são mais difíceis de classificar corretamente.

In [ ]:
import plotly.express as px

# Gráfico de erros por classe
if errors:
    error_data = [{"Classe": cls, "Erros": count} for cls, count in error_by_true.items()]
    fig = px.bar(
        error_data, 
        x="Erros", 
        y="Classe", 
        orientation="h",
        title="Erros por Classe Real",
        color="Erros",
        color_continuous_scale="Reds"
    )
    fig.update_layout(height=350, showlegend=False)
    fig.show()
else:
    print("Nenhum erro de classificação para exibir!")

### Análise de Scores de Similaridade

Comparação dos scores de similaridade entre predições corretas e incorretas. Scores mais baixos em predições incorretas indicam que o retriever teve dificuldade em encontrar exemplos relevantes. Se os scores são similares, o problema está na classificação do LLM.

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Calcular média dos scores de similaridade por predição
def mean_similarity(result):
    scores = [t['score'] for t in result['similar_tickets']]
    return np.mean(scores) if scores else 0

scores_correct = [mean_similarity(r) for r in results if r['correct']]
scores_incorrect = [mean_similarity(r) for r in results if not r['correct']]

# Box plot comparativo
fig = go.Figure()
fig.add_trace(go.Box(y=scores_correct, name='Corretos', marker_color='#2ca02c'))
fig.add_trace(go.Box(y=scores_incorrect, name='Incorretos', marker_color='#d62728'))

fig.update_layout(
    title='Distribuição de Scores de Similaridade (Média dos K Similares)',
    yaxis_title='Score de Similaridade',
    height=400
)
fig.show()

# Estatísticas
print(f"Corretos:   média={np.mean(scores_correct):.3f}, mediana={np.median(scores_correct):.3f}, n={len(scores_correct)}")
if scores_incorrect:
    print(f"Incorretos: média={np.mean(scores_incorrect):.3f}, mediana={np.median(scores_incorrect):.3f}, n={len(scores_incorrect)}")
else:
    print("Nenhuma predição incorreta para análise.")